## В этом домашнем задании вы сделаете первые шаги в мире линейной бинарной классификации!

In [169]:
import pandas as pd
import numpy as np
import warnings

#### Задание 1

Мы будем работать с данными **Microsoft Malware Detection**

Таргетом будет последний столбец `HasDetection`, который принимает значения $\{0,\, 1\}$ в случае отсутствия или наличия вируса на компьютере соответственно. Признаками будут выступать всевозможные характеристики компьютера.

In [170]:
data = pd.read_csv('train.csv')

C:\Users\realn\AppData\Local\Temp\ipykernel_39164\2815211952.py:1: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('train.csv')


In [171]:
data.head()

,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,...,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,0000028988387b115f69f31a3bf04f09,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1735.0,0,7.0,0,NaN,53447.0,...,36144.0,0,NaN,0.0,0,0,0.0,0.0,10.0,0
1,000007535c3f730efa9ea0b7ef1bd645,win8defender,1.1.14600.4,4.13.17134.1,1.263.48.0,0,7.0,0,NaN,53447.0,...,57858.0,0,NaN,0.0,0,0,0.0,0.0,8.0,0
2,000007905a28d863f6d0d597892cd692,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1341.0,0,7.0,0,NaN,53447.0,...,52682.0,0,NaN,0.0,0,0,0.0,0.0,3.0,0
3,00000b11598a75ea8ba1beea8459149f,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1527.0,0,7.0,0,NaN,53447.0,...,20050.0,0,NaN,0.0,0,0,0.0,0.0,3.0,1
4,000014a5f00daa18e76b81417eeb99fc,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1379.0,0,7.0,0,NaN,53447.0,...,19844.0,0,0.0,0.0,0,0,0.0,0.0,1.0,1


Удалите константные признаки и признаки `ProductName` `MachineIdentifier`

In [172]:
### Your code is here
data = data.drop(['ProductName','MachineIdentifier'], axis = 1)






In [187]:
from sklearn.feature_selection import VarianceThreshold

numeric_columns = data.loc[:,data.dtypes!=object].columns
cutter = VarianceThreshold(threshold=0)
cutter.fit(data[numeric_columns])

#cutter.get_feature_names_out() - numeric_columns
data = data.drop(list(set(numeric_columns).difference(set(cutter.get_feature_names_out()))), axis=1)

Посмотрите на соотношение классов в таргете. Все ли хорошо?

In [188]:
print(sum(data['HasDetections'] == 1), '- positive class,')
print(sum(data['HasDetections'] == 0), '- negative class')

100060 - positive class,
99940 - negative class


Ответьте на вопрос: почему с вашей точки зрения важно иметь представление о балансе классов в ваших данных?

Избавьтесь от пропусков в данных! 

Новый для нас прием: признаки с более чем половиной пропусков следует удалить.

Согласитесь, если в вашей колонке среди 100 объектов всего лишь у 2 есть какое-то непропущенное значение, странно все остальные заполнять средним от этих двух чисел. Такие "редкие" признаки лучше вообще опустить!


В категориальных колонках заменим отсутствующую категорию просто некоторой новой и назовем ее `NaN`.

А в числовых, ради разнообразия, заполним пропуски медианным значением.

In [189]:
data.describe()

,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,...,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
count,199236.000000,200000.000000,9703.000000,199196.000000,199196.000000,199196.000000,200000.00000,200000.000000,192785.000000,138463.000000,...,195899.000000,196414.000000,200000.000000,199652.000000,200000.000000,200000.000000,198379.000000,193263.000000,193263.000000,200000.000000
mean,6.848175,0.017090,1654.631969,47867.326116,1.324916,1.021140,0.98768,107.989305,81288.197287,24.870919,...,402.358486,33015.526770,0.485065,0.007022,0.125465,0.037850,0.057249,0.282589,7.877649,0.500300
std,1.016713,0.129607,996.743491,14003.576777,0.521275,0.166286,0.11031,62.960237,48928.959786,5.602695,...,221.782033,21220.733321,0.499778,0.083504,0.331246,0.190834,0.232318,0.450259,4.555336,0.500001
min,0.000000,0.000000,1.000000,39.000000,1.000000,0.000000,0.00000,1.000000,7.000000,1.000000,...,9.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,7.000000,0.000000,788.000000,49480.000000,1.000000,1.000000,1.00000,51.000000,36825.000000,18.000000,...,142.000000,13156.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000
50%,7.000000,0.000000,1632.000000,53447.000000,1.000000,1.000000,1.00000,97.000000,82373.000000,27.000000,...,500.000000,33070.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,1.000000
75%,7.000000,0.000000,2373.000000,53447.000000,2.000000,1.000000,1.00000,160.000000,123975.000000,27.000000,...,556.000000,52436.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,11.000000,1.000000
max,8.000000,1.000000,3204.000000,70486.000000,5.000000,4.000000,1.00000,222.000000,167962.000000,52.000000,...,1087.000000,72091.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,15.000000,1.000000


In [190]:
data.isna().sum()

EngineVersion                                 0
AppVersion                                    0
AvSigVersion                                  0
RtpStateBitfield                            764
IsSxsPassiveMode                              0
                                           ... 
Census_IsPenCapable                           0
Census_IsAlwaysOnAlwaysConnectedCapable    1621
Wdft_IsGamer                               6737
Wdft_RegionIdentifier                      6737
HasDetections                                 0
Length: 79, dtype: int64

In [149]:
len(data.columns)


81

In [154]:
for col in data.columns:
    
    ### К колонкам с маленькой размерностью применим one-hot
    if (data.shape[0]/2) < data[col].isna().sum():
        print(col)


DefaultBrowsersIdentifier
PuaMode
Census_ProcessorClass
Census_InternalBatteryType
Census_IsFlightingInternal
Census_ThresholdOptIn
Census_IsWIMBootEnabled


In [191]:
### Your code is here
for col in data.columns:
    
    ### К колонкам с маленькой размерностью применим one-hot
    if (data.shape[0]/2) < data[col].isna().sum():
        data = data.drop([col], axis = 1)


data.shape




(200000, 73)

In [192]:
numeric_columns = data.loc[:,data.dtypes!=object].columns
categorical_columns = data.loc[:,data.dtypes==object].columns

In [161]:
numeric_columns
categorical_columns

Index(['EngineVersion', 'AppVersion', 'AvSigVersion', 'Platform', 'Processor',
       'OsVer', 'OsPlatformSubRelease', 'OsBuildLab', 'SkuEdition',
       'SmartScreen', 'Census_MDC2FormFactor', 'Census_DeviceFamily',
       'Census_PrimaryDiskTypeName', 'Census_ChassisTypeName',
       'Census_PowerPlatformRoleName', 'Census_OSVersion',
       'Census_OSArchitecture', 'Census_OSBranch', 'Census_OSEdition',
       'Census_OSSkuName', 'Census_OSInstallTypeName',
       'Census_OSWUAutoUpdateOptionsName', 'Census_GenuineStateName',
       'Census_ActivationChannel', 'Census_FlightRing'],
      dtype='object')

In [193]:
data[numeric_columns].describe()
data[numeric_columns].isna().sum()

RtpStateBitfield                                       764
IsSxsPassiveMode                                         0
AVProductStatesIdentifier                              804
AVProductsInstalled                                    804
AVProductsEnabled                                      804
HasTpm                                                   0
CountryIdentifier                                        0
CityIdentifier                                        7215
OrganizationIdentifier                               61537
GeoNameIdentifier                                        1
LocaleEnglishNameIdentifier                              0
OsBuild                                                  0
OsSuite                                                  0
IsProtected                                            799
AutoSampleOptIn                                          0
SMode                                                11863
IeVerIdentifier                                       13

In [194]:
data[categorical_columns].describe()
data[categorical_columns].isna().sum()

EngineVersion                           0
AppVersion                              0
AvSigVersion                            0
Platform                                0
Processor                               0
OsVer                                   0
OsPlatformSubRelease                    0
OsBuildLab                              1
SkuEdition                              0
SmartScreen                         71123
Census_MDC2FormFactor                   0
Census_DeviceFamily                     0
Census_PrimaryDiskTypeName            292
Census_ChassisTypeName                  9
Census_PowerPlatformRoleName            0
Census_OSVersion                        0
Census_OSArchitecture                   0
Census_OSBranch                         0
Census_OSEdition                        0
Census_OSSkuName                        0
Census_OSInstallTypeName                0
Census_OSWUAutoUpdateOptionsName        0
Census_GenuineStateName                 0
Census_ActivationChannel          

In [195]:
for col in numeric_columns:
    data[col] = data[col].fillna(data[col].median())

In [196]:
for col in categorical_columns:
    data[col] = data[col].fillna('NaN')

In [197]:
data

,EngineVersion,AppVersion,AvSigVersion,RtpStateBitfield,IsSxsPassiveMode,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,...,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,1.1.15100.1,4.18.1807.18075,1.273.1735.0,7.0,0,53447.0,1.0,1.0,1,29,...,628.0,36144.0,0,0.0,0,0,0.0,0.0,10.0,0
1,1.1.14600.4,4.13.17134.1,1.263.48.0,7.0,0,53447.0,1.0,1.0,1,93,...,628.0,57858.0,0,0.0,0,0,0.0,0.0,8.0,0
2,1.1.15100.1,4.18.1807.18075,1.273.1341.0,7.0,0,53447.0,1.0,1.0,1,86,...,142.0,52682.0,0,0.0,0,0,0.0,0.0,3.0,0
3,1.1.15100.1,4.18.1807.18075,1.273.1527.0,7.0,0,53447.0,1.0,1.0,1,88,...,355.0,20050.0,0,0.0,0,0,0.0,0.0,3.0,1
4,1.1.15100.1,4.18.1807.18075,1.273.1379.0,7.0,0,53447.0,1.0,1.0,1,18,...,355.0,19844.0,0,0.0,0,0,0.0,0.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,1.1.15100.1,4.18.1807.18075,1.273.894.0,0.0,1,3371.0,2.0,1.0,1,160,...,628.0,28487.0,1,0.0,0,0,0.0,0.0,3.0,0
199996,1.1.15200.1,4.18.1807.18075,1.275.1234.0,7.0,0,53447.0,1.0,1.0,1,60,...,142.0,46942.0,0,0.0,0,0,1.0,0.0,15.0,0
199997,1.1.15000.2,4.18.1806.18062,1.271.1198.0,7.0,0,62773.0,1.0,1.0,1,9,...,142.0,12463.0,1,0.0,0,0,0.0,1.0,10.0,0
199998,1.1.15100.1,4.11.15063.447,1.273.438.0,7.0,0,49480.0,2.0,1.0,1,158,...,513.0,63437.0,1,0.0,0,0,1.0,0.0,1.0,1


Создайте копию полученного датафрейма и положите ее в переменную data_2. Понадобится в следующих заданиях.

In [198]:
data_2 = data.copy()
data_2.shape

(200000, 73)

Так же поработаем над всеми категориальными колонкам перед запуском непосредственно моделей.

Провернем самый базовый и наглый метод - несмотря на количество уникальных значений в каждой категории, просто применим ко всей категориальной части датасета `OneHotEncoding`

In [199]:
### Your code is here

for col in categorical_columns:
    
    ### К колонкам с маленькой размерностью применим one-hot
    one_hot = pd.get_dummies(data[col], prefix=col, drop_first=True)
    data = pd.concat((data.drop(col, axis=1), one_hot), axis=1)

data.shape





(200000, 6114)

###  Разделим выборку на тренировочную и тестовую

P.S. в задачах классификации, как и в задаче регрессии, можно использовать технологию Кросс-Валидации. Например, по одному из двух следующих сценариев:

1) Отделить валидацию и тест, произвести подбор лучшей модели с помощью `K-Fold` на валидации, финально обучить выбранную модель на всей валидации и замерить качество на заранее отложенном финальном тесте!

2) Всю выборку назвать валидационной и на ней применить `K-Fold` без финального замера.

В этой домашней работе попросим Вас быть еще проще! :)
Реализуем просто технологию отложенной выборки в пропорции 3:1

In [200]:
from sklearn.model_selection import train_test_split 

X = data.drop(columns=['HasDetections'])
y = data['HasDetections']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.25,
                                                    shuffle=True,
                                                    random_state=1)

Соберите `Pipeline`, реализовав в нем 2 шага: стандартизация данных через `StandardScaler` и обучение логистической регрессии с помощью `LogisticRegression`, положите результаты в переменную `pipe`, а в классе модели `LogisticRegression` укажите параметр `penalty='none'`

In [202]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

### Your code is here
pipe = Pipeline([('scaler', StandardScaler()),
                 ('LR', LogisticRegression(penalty='none'))])  # убираем регуляризацию

pipe.fit(X_train, y_train)





MemoryError: Unable to allocate 6.83 GiB for an array with shape (150000, 6113) and data type float64

Чтобы замерить качество работы такой модели на трейне и на тесте воспользуемся функцией `cross_validate`

Вопрос: что передавать ей в параметр cv? Ведь мы уже разделили нашу выборку на трейн и тест (имеем всего 1 fold). Для этого можно просто передать список от кортежа, содержащего индексы тренировочных и тестовых объектов.

In [ ]:
from sklearn.model_selection import cross_validate
import datetime

custom_cv = [(X_train.index.to_list(), X_test.index.to_list())]

begin_time = datetime.datetime.now()

cv_result_pipe = cross_validate(pipe, X, y, scoring='accuracy',
                                cv=custom_cv, return_train_score=True)


print(f"Accuracy на трейне: {np.mean(cv_result_pipe['train_score']).round(3)}")
print(f"Accuracy на тесте: {np.mean(cv_result_pipe['test_score']).round(3)}")

print(f"Время работы алгоритма: {datetime.datetime.now() - begin_time}")

Что можете сказать про время работы алгоритма?

Очевидно, оно достаточно большие. Уж тем более для линейных моделей.

Такое как раз-таки происходит из-за того, что количество фичей, который мы передали нашей модели - гигантское! Классу требуется много времени и памяти, чтобы обработать датасет.

Поэтому те колонки, в которых количество уникальных категорий превышает какое-то адекватное число, следует кодировать иначе, нежели с помощью технологии `One-Hot-Encoding`.

Теперь вы верите, что более умные кодировки зачастую прям необходимы! Раньше мы этот факт не демонстрировали!

Дело еще вот в чем: в классе `LogisticRegression`, как и, например, `Lasso`, есть параметр, ограничивающий максимальное количество итераций во время обучения модели. Так, если данных много и итераций тоже ожидается большое число, найденная разделяющая гиперплоскость может оказаться не самой лучшей, так как наш алгоритм (будь то градиентный спуск или любой иной) просто 'не доползет'. 

#### Задание 2

Теперь попробуем другой метод кодирования категориальных колонок, а именно счётчики.
Построем ту же модель и на том же разделении, просто заново иначе переобработаем датасет. 

Для тех категориальных признаков, у которых количество уникальных значений в колоночках больше 5, применим `MeanTargetEncoding`.

Для остальных оставим любимый `OneHotEncoding` (как делали на практике и в предыдущем уроке).

In [203]:
### Your code is here
for col in categorical_columns:
    
    ### К колонкам с маленькой размерностью применим one-hot
    if data_2[col].nunique() <= 5:
        one_hot = pd.get_dummies(data_2[col], prefix=col, drop_first=True)
        data_2 = pd.concat((data_2.drop(col, axis=1), one_hot), axis=1)
        
    ### К остальным - счетчики
    else:
        mean_target = data_2.groupby(col)['HasDetections'].mean()
        data_2[col] = data_2[col].map(mean_target)




In [204]:
X_2 = data_2.drop(columns=['HasDetections'])
y_2 = data_2['HasDetections']

Опять обучим модель, пока что без изменений! Скажите, стало ли быстрее? А что с качеством?

In [206]:
### Your code is here
### Your code is here
from sklearn.model_selection import cross_validate
import datetime
X = data_2.drop(columns=['HasDetections'])
y = data_2['HasDetections']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.25,
                                                    shuffle=True,
                                                    random_state=1)
pipe = Pipeline([('scaler', StandardScaler()),
                 ('LR', LogisticRegression(penalty='none'))])  # убираем регуляризацию

pipe.fit(X_train, y_train)

custom_cv = [(X_train.index.to_list(), X_test.index.to_list())]

begin_time = datetime.datetime.now()

cv_result_pipe = cross_validate(pipe, X, y, scoring='accuracy',
                                cv=custom_cv, return_train_score=True)


print(f"Accuracy на трейне: {np.mean(cv_result_pipe['train_score']).round(3)}")
print(f"Accuracy на тесте: {np.mean(cv_result_pipe['test_score']).round(3)}")

print(f"Время работы алгоритма: {datetime.datetime.now() - begin_time}")

Accuracy на трейне: 0.638
Accuracy на тесте: 0.638
Время работы алгоритма: 0:00:02.535505


#### Задание 3: Регуляризация

Как и в моделях регрессии, решая задачу классификации, можем штрафовать минимизируемый функционал за большие веса, добавив к нему L1 или L2 норму весов (все как раньше!).

Для этого в изначальном классе `LogisticRegression` изменить параметр `penalty` на l1 или l2 соответственно. Выберите второй вариант! Можно воспользоваться методом `set_params` и применить его к `pipe`.

In [208]:
### Your code is here
pipe.set_params(LR__penalty='l2')


Pipeline(steps=[('scaler', StandardScaler()), ('LR', LogisticRegression())])

In [209]:
pipe.get_params()

{'memory': None,
 'steps': [('scaler', StandardScaler()), ('LR', LogisticRegression())],
 'verbose': False,
 'scaler': StandardScaler(),
 'LR': LogisticRegression(),
 'scaler__copy': True,
 'scaler__with_mean': True,
 'scaler__with_std': True,
 'LR__C': 1.0,
 'LR__class_weight': None,
 'LR__dual': False,
 'LR__fit_intercept': True,
 'LR__intercept_scaling': 1,
 'LR__l1_ratio': None,
 'LR__max_iter': 100,
 'LR__multi_class': 'auto',
 'LR__n_jobs': None,
 'LR__penalty': 'l2',
 'LR__random_state': None,
 'LR__solver': 'lbfgs',
 'LR__tol': 0.0001,
 'LR__verbose': 0,
 'LR__warm_start': False}

Теперь наша модель будет строить логистическую регрессию с L2 регуляризатором! Помним, что у регуляризируемых моделей есть гиперпараметр, контролирующий силу регуляризации, который выбирается ДО запуска метода fit, то есть заранее, когда модель еще не обучена. 

Конечно же, выбор этого параметра влияет итоговые результаты. Хочется поперебирать различные параметры регуляризации и найти такой, при котором качество на тесте окажется лучшим! 

Сгенерируем массив гиперпараметра, которые планируем перебрать:

In [210]:
alphas = np.linspace(0.01, 100, 100)
alphas

array([1.000e-02, 1.020e+00, 2.030e+00, 3.040e+00, 4.050e+00, 5.060e+00,
       6.070e+00, 7.080e+00, 8.090e+00, 9.100e+00, 1.011e+01, 1.112e+01,
       1.213e+01, 1.314e+01, 1.415e+01, 1.516e+01, 1.617e+01, 1.718e+01,
       1.819e+01, 1.920e+01, 2.021e+01, 2.122e+01, 2.223e+01, 2.324e+01,
       2.425e+01, 2.526e+01, 2.627e+01, 2.728e+01, 2.829e+01, 2.930e+01,
       3.031e+01, 3.132e+01, 3.233e+01, 3.334e+01, 3.435e+01, 3.536e+01,
       3.637e+01, 3.738e+01, 3.839e+01, 3.940e+01, 4.041e+01, 4.142e+01,
       4.243e+01, 4.344e+01, 4.445e+01, 4.546e+01, 4.647e+01, 4.748e+01,
       4.849e+01, 4.950e+01, 5.051e+01, 5.152e+01, 5.253e+01, 5.354e+01,
       5.455e+01, 5.556e+01, 5.657e+01, 5.758e+01, 5.859e+01, 5.960e+01,
       6.061e+01, 6.162e+01, 6.263e+01, 6.364e+01, 6.465e+01, 6.566e+01,
       6.667e+01, 6.768e+01, 6.869e+01, 6.970e+01, 7.071e+01, 7.172e+01,
       7.273e+01, 7.374e+01, 7.475e+01, 7.576e+01, 7.677e+01, 7.778e+01,
       7.879e+01, 7.980e+01, 8.081e+01, 8.182e+01, 

Чтобы отобрать среди данного массива гиперпараметров лучший, воспользуйтесь конструкцией `GridSearchCV` из `sklearn`

In [213]:
from sklearn.model_selection import GridSearchCV

param_grid ={
    'LR__C': alphas
}

### Your code is here
search = GridSearchCV(pipe, param_grid, cv=custom_cv, scoring='accuracy')

search.fit(X, y)

print(f"Best parameter (CV score={search.best_score_:.5f}):")
print(search.best_params_)

Best parameter (CV score=0.63800):
{'LR__C': 0.01}


#### Задание 4: Бонус

Как вы знаете, подбор признаков является одной из самых важных частей машинного обучения. Сейчас вы попробуете на основе имеющихся признаков сгенерировать новые. В качестве новых признаков будем использовать мономы 2 степени. Можно использовать регуляризацию различного рода, выбор энкодера тоже за вами. Ваша задача - добиться качества `0.65` на тестовой выборке

In [19]:
### Your code is here

